In [6]:
# OPZIONE VALIDA(? DUE PESI CHE RAPPRESENTANO LO STESSO PESO POTREBBERO ESSERE DIVERSI FRA LORO), MA MOLTO SCOMODA
import torch
import torch.nn as nn
import torch.optim as optim

# Rete 1: Un neurone di input connesso a due neuroni di output
class SubNet1(nn.Module):
    def __init__(self):
        super(SubNet1, self).__init__()
        self.fc = nn.Linear(1, 2, bias=False)  # 1 input, 2 output

    def forward(self, x):
        return self.fc(x)

# Rete 2: Un neurone di input connesso a un neurone di output
class SubNet2(nn.Module):
    def __init__(self):
        super(SubNet2, self).__init__()
        self.fc = nn.Linear(1, 1, bias=False)  # 1 input, 1 output

    def forward(self, x):
        return self.fc(x)

# Rete 3: Un neurone di input connesso a un neurone di output
class SubNet3(nn.Module):
    def __init__(self):
        super(SubNet3, self).__init__()
        self.fc = nn.Linear(1, 1, bias=False)  # 1 input, 1 output

    def forward(self, x):
        return self.fc(x)

# Inizializza le sotto-reti
net1 = SubNet1()
net2 = SubNet2()
net3 = SubNet3()

# Esempio di input
x = torch.tensor([[1.0, 2.0, 3.0]], requires_grad=False)
target = torch.tensor([[0.5, 0.5]], requires_grad=False)

# Ottimizzatori per ciascuna sotto-rete
optimizer1 = optim.SGD(net1.parameters(), lr=0.01)
optimizer2 = optim.SGD(net2.parameters(), lr=0.01)
optimizer3 = optim.SGD(net3.parameters(), lr=0.01)

# Loss function
criterion = nn.MSELoss()

# Addestramento
for epoch in range(10):
    # Azzerare i gradienti
    optimizer1.zero_grad()
    optimizer2.zero_grad()
    optimizer3.zero_grad()

    # Forward pass
    out1 = net1(x[:, 0:1])  # Usa solo il primo input per la rete 1
    out2 = net2(x[:, 1:2])  # Usa solo il secondo input per la rete 2
    out3 = net3(x[:, 2:3])  # Usa solo il terzo input per la rete 3

    # y_pred1 è la somma dell'output della rete 1 e della rete 2
    y_pred1 = out1[:, 0:1] + out2
    # y_pred2 è la somma dell'output della rete 1 e della rete 3
    y_pred2 = out1[:, 1:2] + out3

    # Combina i risultati per ottenere l'output finale
    output = torch.cat([y_pred1, y_pred2], dim=1)

    # Calcola la loss
    loss = criterion(output, target)

    # Backward pass
    loss.backward()

    # Ottimizzazione
    optimizer1.step()
    optimizer2.step()
    optimizer3.step()

    # Stampa i pesi e la loss
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")
    print("Pesi SubNet1:", list(net1.parameters()))
    print("Pesi SubNet2:", list(net2.parameters()))
    print("Pesi SubNet3:", list(net3.parameters()))


Epoch 1, Loss: 6.048243045806885
Pesi SubNet1: [Parameter containing:
tensor([[ 0.2880],
        [-0.1509]], requires_grad=True)]
Pesi SubNet2: [Parameter containing:
tensor([[0.5428]], requires_grad=True)]
Pesi SubNet3: [Parameter containing:
tensor([[-0.7893]], requires_grad=True)]
Epoch 2, Loss: 4.938197135925293
Pesi SubNet1: [Parameter containing:
tensor([[ 0.2793],
        [-0.1207]], requires_grad=True)]
Pesi SubNet2: [Parameter containing:
tensor([[0.5254]], requires_grad=True)]
Pesi SubNet3: [Parameter containing:
tensor([[-0.6987]], requires_grad=True)]
Epoch 3, Loss: 4.0352463722229
Pesi SubNet1: [Parameter containing:
tensor([[ 0.2710],
        [-0.0935]], requires_grad=True)]
Pesi SubNet2: [Parameter containing:
tensor([[0.5088]], requires_grad=True)]
Pesi SubNet3: [Parameter containing:
tensor([[-0.6172]], requires_grad=True)]
Epoch 4, Loss: 3.3004138469696045
Pesi SubNet1: [Parameter containing:
tensor([[ 0.2631],
        [-0.0691]], requires_grad=True)]
Pesi SubNet2: [P

In [6]:
# METODO MIGLIORE CHE ESISTA
import torch
import torch.nn as nn
import torch.optim as optim

# Definizione del modello
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc = nn.Linear(3, 2, bias=False)  # 3 input, 2 output, no bias

    def forward(self, x):
        return self.fc(x)

# Funzione di aggiornamento manuale
def manual_update(model, learning_rate, mask=None):
    with torch.no_grad():
        for param in model.parameters():
            if param.grad is not None:
                if mask is not None:
                    param.grad *= mask
                param -= learning_rate * param.grad

# Inizializzazione del modello
model = SimpleNet()

# Definizione della funzione di perdita
criterion = nn.MSELoss()

# Creazione della maschera (ad esempio, per congelare il primo e il terzo peso della prima colonna)
mask = torch.tensor([[0, 1, 0],  # Maschera per il layer 0
                     [0, 1, 0]], dtype=torch.float32)

# Definizione del learning rate
learning_rate = 0.01

# Dati di esempio per l'addestramento
inputs = torch.tensor([[1.0, 2.0, 3.0],
                       [4.0, 5.0, 6.0],
                       [7.0, 8.0, 9.0],
                       [10.0, 11.0, 12.0]])

targets = torch.tensor([[0.5, 0.5],
                        [1.0, 1.0],
                        [1.5, 1.5],
                        [2.0, 2.0]])

# Numero di epoche di addestramento
num_epochs = 10

# Processo di addestramento
for epoch in range(num_epochs):
    # Azzerare i gradienti
    model.zero_grad()

    # Passaggio avanti
    outputs = model(inputs)

    # Calcolare la loss
    loss = criterion(outputs, targets)

    # Passaggio indietro (calcolare i gradienti)
    loss.backward()

    # Aggiornare i pesi manualmente
    manual_update(model, learning_rate, mask)

    # Stampa della loss e dei pesi ogni epoca
    if (epoch+1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        print("Pesi della rete:", list(model.parameters()))


Epoch [1/10], Loss: 8.1044
Pesi della rete: [Parameter containing:
tensor([[ 0.3642,  0.1335, -0.4391],
        [-0.3598,  0.4738, -0.0783]], requires_grad=True)]
Epoch [2/10], Loss: 1.8310
Pesi della rete: [Parameter containing:
tensor([[ 0.3642,  0.2543, -0.4391],
        [-0.3598,  0.5364, -0.0783]], requires_grad=True)]
Epoch [3/10], Loss: 0.4745
Pesi della rete: [Parameter containing:
tensor([[ 0.3642,  0.3105, -0.4391],
        [-0.3598,  0.5655, -0.0783]], requires_grad=True)]
Epoch [4/10], Loss: 0.1812
Pesi della rete: [Parameter containing:
tensor([[ 0.3642,  0.3367, -0.4391],
        [-0.3598,  0.5790, -0.0783]], requires_grad=True)]
Epoch [5/10], Loss: 0.1178
Pesi della rete: [Parameter containing:
tensor([[ 0.3642,  0.3488, -0.4391],
        [-0.3598,  0.5853, -0.0783]], requires_grad=True)]
Epoch [6/10], Loss: 0.1041
Pesi della rete: [Parameter containing:
tensor([[ 0.3642,  0.3545, -0.4391],
        [-0.3598,  0.5882, -0.0783]], requires_grad=True)]
Epoch [7/10], Loss: 0.

In [4]:
for param in model.parameters():
    print(param.grad)

tensor([[ 0.0000, -0.0147, -0.0000],
        [ 0.0000,  0.0430,  0.0000]])


In [5]:
# ESPERIMENTO 626

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# Definizione del modello
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc = nn.Linear(3, 2, bias=False)  # 3 input, 2 output, no bias

    def forward(self, x):
        return self.fc(x)

# Funzione di perdita personalizzata (mean squared error)
def custom_loss(output, target):
    return torch.mean((output - target) ** 2)

# Funzione per calcolare manualmente i gradienti e applicare la maschera
def manual_backward(model, criterion, inputs, targets, mask):
    # Passaggio avanti
    outputs = model(inputs)
    
    # Calcolare la perdita
    loss = criterion(outputs, targets)
    
    # Calcolare la perdita rispetto agli output
    loss_grad = 2 * (outputs - targets) / outputs.size(0)
    
    # Ottieni i pesi del modello
    weights = model.fc.weight
    
    
    # Calcolare la derivata della loss rispetto ai pesi manualmente
    weight_derivatives = torch.zeros_like(weights) #Crea un tensore di gradienti della stessa forma dei pesi, inizialmente riempito di zeri
    for i in range(weights.size(0)):  # per ogni neurone di output (numero di righe nella matrice W)
        for j in range(weights.size(1)):  # per ogni neurone di input
            if mask[i][j] != 0:
                weight_derivatives[i, j] =  (2 / inputs.size(0)) * torch.sum(inputs[:, j] * loss_grad[:, i])
            else:
                weight_derivatives[i, j] = 0
            
            
    
    # Applicare la maschera ai gradienti
    weight_derivatives *= mask
    
    return loss, weight_derivatives

# Funzione di aggiornamento manuale dei pesi
def manual_update(model, gradients, learning_rate):
    with torch.no_grad():
        model.fc.weight -= learning_rate * gradients
        model.fc.weight *= mask  # Imposta a 0 i pesi non aggiornati
        
# Inizializzazione del modello
model = SimpleNet()

# Definizione della funzione di perdita
criterion = custom_loss

# Creazione della maschera (ad esempio, per congelare il primo e il terzo peso della prima colonna)
mask = torch.tensor([[0, 1, 0],  # Maschera per il layer 0
                     [0, 1, 0]], dtype=torch.float32)

# Definizione del learning rate
learning_rate = 0.01

# Dati di esempio per l'addestramento
inputs = torch.tensor([[1.0, 2.0, 3.0],
                       [4.0, 5.0, 6.0],
                       [7.0, 8.0, 9.0],
                       [10.0, 11.0, 12.0]])

targets = torch.tensor([[0.5, 0.5],
                        [1.0, 1.0],
                        [1.5, 1.5],
                        [2.0, 2.0]])

# Numero di epoche di addestramento
num_epochs = 10

# Processo di addestramento
for epoch in range(num_epochs):
    # Calcolare la loss e i gradienti manualmente
    loss, gradients = manual_backward(model, criterion, inputs, targets, mask)
    
    # Aggiornare i pesi manualmente
    manual_update(model, gradients, learning_rate)
    
    # Stampa della loss e dei pesi ogni epoca
    if (epoch+1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        print("Pesi della rete:", list(model.parameters()))

# ESPERIMENTO RIUSCITO

Epoch [1/10], Loss: 33.2715
Pesi della rete: [Parameter containing:
tensor([[ 0.0000, -0.0497, -0.0000],
        [-0.0000,  0.0313, -0.0000]], requires_grad=True)]
Epoch [2/10], Loss: 2.1518
Pesi della rete: [Parameter containing:
tensor([[0.0000, 0.0769, -0.0000],
        [-0.0000, 0.1146, -0.0000]], requires_grad=True)]
Epoch [3/10], Loss: 0.4699
Pesi della rete: [Parameter containing:
tensor([[0.0000, 0.1357, -0.0000],
        [-0.0000, 0.1533, -0.0000]], requires_grad=True)]
Epoch [4/10], Loss: 0.1062
Pesi della rete: [Parameter containing:
tensor([[0.0000, 0.1631, -0.0000],
        [-0.0000, 0.1713, -0.0000]], requires_grad=True)]
Epoch [5/10], Loss: 0.0275
Pesi della rete: [Parameter containing:
tensor([[0.0000, 0.1759, -0.0000],
        [-0.0000, 0.1796, -0.0000]], requires_grad=True)]
Epoch [6/10], Loss: 0.0105
Pesi della rete: [Parameter containing:
tensor([[0.0000, 0.1818, -0.0000],
        [-0.0000, 0.1835, -0.0000]], requires_grad=True)]
Epoch [7/10], Loss: 0.0069
Pesi dell

In [20]:
# AGGIUNTA DEL BIAS
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# Definizione del modello
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc = nn.Linear(3, 2, bias=True)  # 3 input, 2 output, con bias

    def forward(self, x):
        return self.fc(x)

# Funzione di perdita personalizzata (mean squared error)
def custom_loss(output, target):
    return torch.mean((output - target) ** 2)

# Funzione per calcolare manualmente i gradienti e applicare la maschera
def manual_backward(model, criterion, inputs, targets, mask):
    # Passaggio avanti
    outputs = model(inputs)
    
    # Calcolare la perdita
    loss = criterion(outputs, targets)
    
    # Calcolare la perdita rispetto agli output
    loss_grad = 2 * (outputs - targets) / outputs.size(0)
    
    # Ottieni i pesi e il bias del modello
    weights = model.fc.weight
    biases = model.fc.bias
    
    # Calcolare la derivata della loss rispetto ai pesi manualmente
    weight_derivatives = torch.zeros_like(weights)  # Crea un tensore di gradienti della stessa forma dei pesi, inizialmente riempito di zeri
    bias_derivatives = torch.zeros_like(biases)  # Gradienti per il bias
    
    for i in range(weights.size(0)):  # per ogni neurone di output (numero di righe nella matrice W)
        for j in range(weights.size(1)):  # per ogni neurone di input
            if mask[i][j] != 0:
                weight_derivatives[i, j] = (2 / inputs.size(0)) * torch.sum(inputs[:, j] * loss_grad[:, i])
            else:
                weight_derivatives[i, j] = 0
        
        # Calcola il gradiente rispetto al bias per ogni neurone di output
        bias_derivatives[i] = (2 / inputs.size(0)) * torch.sum(loss_grad[:, i])
    
    # Applicare la maschera ai gradienti dei pesi
    weight_derivatives *= mask
    
    return loss, weight_derivatives, bias_derivatives

# Funzione di aggiornamento manuale dei pesi e del bias
def manual_update(model, weight_gradients, bias_gradients, learning_rate):
    with torch.no_grad():
        model.fc.weight -= learning_rate * weight_gradients
        model.fc.bias -= learning_rate * bias_gradients
        model.fc.weight *= mask  # Imposta a 0 i pesi non aggiornati (secondo la maschera)
        
# Inizializzazione del modello
model = SimpleNet()

# Definizione della funzione di perdita
criterion = custom_loss

# Creazione della maschera (ad esempio, per congelare il primo e il terzo peso della prima colonna)
mask = torch.tensor([[0, 1, 0],  # Maschera per il layer 0
                     [0, 1, 0]], dtype=torch.float32)

# Definizione del learning rate
learning_rate = 0.01

# Dati di esempio per l'addestramento
inputs = torch.tensor([[1.0, 2.0, 3.0],
                       [4.0, 5.0, 6.0],
                       [7.0, 8.0, 9.0],
                       [10.0, 11.0, 12.0]])

targets = torch.tensor([[0.5, 0.5],
                        [1.0, 1.0],
                        [1.5, 1.5],
                        [2.0, 2.0]])

# Numero di epoche di addestramento
num_epochs = 10

# Processo di addestramento
for epoch in range(num_epochs):
    # Calcolare la loss e i gradienti manualmente
    loss, weight_gradients, bias_gradients = manual_backward(model, criterion, inputs, targets, mask)
    
    # Aggiornare i pesi e il bias manualmente
    manual_update(model, weight_gradients, bias_gradients, learning_rate)
    
    # Stampa della loss e dei pesi ogni epoca
    if (epoch+1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        print("Pesi della rete:", list(model.parameters()))


Epoch [1/10], Loss: 26.8107
Pesi della rete: [Parameter containing:
tensor([[-0.0000, 0.2608, 0.0000],
        [0.0000, 0.0042, -0.0000]], requires_grad=True), Parameter containing:
tensor([ 0.0293, -0.3915], requires_grad=True)]
Epoch [2/10], Loss: 1.6143
Pesi della rete: [Parameter containing:
tensor([[-0.0000, 0.2194, 0.0000],
        [0.0000, 0.1274, -0.0000]], requires_grad=True), Parameter containing:
tensor([ 0.0245, -0.3753], requires_grad=True)]
Epoch [3/10], Loss: 0.3623
Pesi della rete: [Parameter containing:
tensor([[-0.0000, 0.2004, 0.0000],
        [0.0000, 0.1836, -0.0000]], requires_grad=True), Parameter containing:
tensor([ 0.0225, -0.3674], requires_grad=True)]
Epoch [4/10], Loss: 0.1007
Pesi della rete: [Parameter containing:
tensor([[-0.0000, 0.1917, 0.0000],
        [0.0000, 0.2093, -0.0000]], requires_grad=True), Parameter containing:
tensor([ 0.0217, -0.3631], requires_grad=True)]
Epoch [5/10], Loss: 0.0459
Pesi della rete: [Parameter containing:
tensor([[-0.0000

In [21]:
outputs = model(inputs)
print(outputs) # come vedi poi dentro torch le matrici si traspongono e fanno cose strane

tensor([[0.3913, 0.1060],
        [0.9442, 0.7959],
        [1.4970, 1.4858],
        [2.0499, 2.1757]], grad_fn=<AddmmBackward0>)


In [67]:
# PER ORA ER MEJO CHE SI PUO'
import torch
import torch.nn as nn
import numpy as np

# Definizione del modello con hidden layers e bias
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(3, 4, bias=True)  # 3 input, 4 output (hidden layer) con bias
        self.fc2 = nn.Linear(4, 2, bias=True)  # 4 input (hidden layer), 2 output con bias

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # Attivazione ReLU dopo il primo layer
        x = self.fc2(x)
        return x

# Funzione di perdita personalizzata (mean squared error)
def custom_loss(output, target):
    return torch.mean((output - target) ** 2)

# Funzione per calcolare manualmente i gradienti e applicare le maschere
def manual_backward(model, criterion, inputs, targets, masks):
    # Passaggio avanti
    outputs = model(inputs) # outputs è una matrice di dimensione (batch x outputs)
    
    # Calcolare la perdita
    loss = criterion(outputs, targets)
    
    # Calcolare la perdita rispetto agli output
    loss_grad = 2 * (outputs - targets) / outputs.size(0)

    # Calcolare i gradienti per fc2 (secondo layer) (parto da W2)
    weight_derivatives2 = torch.zeros_like(model.fc2.weight)
    bias_derivatives2 = torch.zeros_like(model.fc2.bias)  # Gradienti per il bias del secondo layer
    hidden_outputs = torch.relu(model.fc1(inputs))
    
    for i in range(weight_derivatives2.size(0)):  # per ogni neurone di output
        for j in range(weight_derivatives2.size(1)):  # per ogni neurone di input (hidden layer)
            if masks[1][i][j] != 0:
                weight_derivatives2[i, j] = (2 / inputs.size(0)) * torch.sum(hidden_outputs[:, j] * loss_grad[:, i])
            else:
                weight_derivatives2[i, j] = 0
        # Calcolare i gradienti per il bias
        bias_derivatives2[i] = (2 / inputs.size(0)) * torch.sum(loss_grad[:, i])

    # Calcolare la perdita rispetto al hidden layer
    hidden_loss_grad = torch.matmul(loss_grad, model.fc2.weight) * (hidden_outputs > 0).float()  # ReLU derivative### PRIMA DELLA TUA MODIFICA AVEVI model.fc2.weight().t TRASPOSTA

    # Calcolare i gradienti per fc1 (primo layer)
    weight_derivatives1 = torch.zeros_like(model.fc1.weight)
    bias_derivatives1 = torch.zeros_like(model.fc1.bias)  # Gradienti per il bias del primo layer
    
    for i in range(weight_derivatives1.size(0)):  # per ogni neurone di hidden layer
        for j in range(weight_derivatives1.size(1)):  # per ogni neurone di input
            if masks[0][i][j] != 0:
                weight_derivatives1[i, j] = (2 / inputs.size(0)) * torch.sum(inputs[:, j] * hidden_loss_grad[:, i])
            else:
                weight_derivatives1[i, j] = 0
        # Calcolare i gradienti per il bias
        bias_derivatives1[i] = (2 / inputs.size(0)) * torch.sum(hidden_loss_grad[:, i])

    # Applicare le maschere ai gradienti
    weight_derivatives1 *= masks[0]
    weight_derivatives2 *= masks[1]
    
    return loss, [weight_derivatives1, weight_derivatives2], [bias_derivatives1, bias_derivatives2]

# Funzione di aggiornamento manuale dei pesi e dei bias per tutti i layer
def manual_update(model, weight_gradients, bias_gradients, learning_rate, masks):
    with torch.no_grad():
        # Aggiornamento pesi e bias del primo layer
        model.fc1.weight -= learning_rate * weight_gradients[0]
        model.fc1.bias -= learning_rate * bias_gradients[0]
        model.fc1.weight *= masks[0]  # Imposta a 0 i pesi non aggiornati del primo layer
        
        # Aggiornamento pesi e bias del secondo layer
        model.fc2.weight -= learning_rate * weight_gradients[1]
        model.fc2.bias -= learning_rate * bias_gradients[1]
        model.fc2.weight *= masks[1]  # Imposta a 0 i pesi non aggiornati del secondo layer

# Inizializzazione del modello
model = SimpleNet()

# Definizione della funzione di perdita
criterion = custom_loss

# Creazione delle maschere per ogni layer
mask1 = torch.tensor([[0, 1, 0], [1, 0, 1], [0, 1, 1], [1, 0, 0]], dtype=torch.float32)  # Maschera per il primo layer
mask2 = torch.tensor([[1, 0, 1, 0], [0, 1, 0, 1]], dtype=torch.float32)  # Maschera per il secondo layer
masks = [mask1, mask2]

# Definizione del learning rate
learning_rate = 0.1

# Dati di esempio per l'addestramento
inputs = torch.tensor([[1.0, 2.0, 3.0],
                       [4.0, 5.0, 6.0],
                       [7.0, 8.0, 9.0],
                       [10.0, 11.0, 12.0],
                       [13.0, 14.0, 15.0]])

targets = torch.tensor([[0.5, 1.5],
                        [1.0, 7.0],
                        [6.5, 1.5],
                        [2.0, 13.0],
                        [3.0, 4.0]])

# Numero di epoche di addestramento
num_epochs = 10

# Processo di addestramento
for epoch in range(num_epochs):
    # Calcolare la loss e i gradienti manualmente
    loss, gradients, bias_gradients = manual_backward(model, criterion, inputs, targets, masks)
    
    # Aggiornare i pesi e i bias manualmente
    manual_update(model, gradients, bias_gradients, learning_rate, masks)
    
    # Stampa della loss e dei pesi ogni epoca
    if (epoch+1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        #print("Pesi del primo layer:", model.fc1.weight)
        #print("Bias del primo layer:", model.fc1.bias)
        #rint("Pesi del secondo layer:", model.fc2.weight)
        #print("Bias del secondo layer:", model.fc2.bias)


Epoch [1/10], Loss: 37.3030
Epoch [2/10], Loss: 25.9641
Epoch [3/10], Loss: 23.7486
Epoch [4/10], Loss: 21.8733
Epoch [5/10], Loss: 20.2861
Epoch [6/10], Loss: 18.9427
Epoch [7/10], Loss: 17.8057
Epoch [8/10], Loss: 16.8433
Epoch [9/10], Loss: 16.0287
Epoch [10/10], Loss: 15.3392
